# Notebook 06: Test Approfondi du Pipeline de Bout en Bout

Ce notebook est dédié à un test complet et une analyse détaillée du workflow "Cognitive Swarm" sur une requête utilisateur complexe. Nous allons observer les sorties intermédiaires des agents, le flux de décision, et la qualité de la synthèse finale. Nous explorerons également comment inspecter les états sauvegardés par le checkpointer MongoDB.

**Prérequis :**
* Environnement configuré via `00_setup_environment.ipynb`.
* Base de données MongoDB peuplée via `01_data_ingestion_and_embedding.ipynb` ou `scripts/run_ingestion.py`.
* Clés API (`OPENAI_API_KEY`, `MONGO_URI`) configurées dans `.env`.
* Le `MongoDBSaver` (checkpointer) est activé par défaut dans `src/graph/main_workflow.py`.

In [ ]:
import logging
import sys
from pathlib import Path
import os
import json
import asyncio
import uuid 
import pprint # Pour un affichage plus lisible des dictionnaires

# Ajout de la racine du projet au PYTHONPATH
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    print(f"Ajout de {project_root} au PYTHONPATH")

from dotenv import load_dotenv
dotenv_path = project_root / ".env"
if dotenv_path.exists():
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Variables d'environnement chargées depuis : {dotenv_path}")
else:
    print(f"ATTENTION: Fichier .env non trouvé à {dotenv_path}.")

from config.settings import settings
from config.logging_config import setup_logging

from src.graph.main_workflow import run_cognitive_swarm_v2_1, GraphState # GraphState pour type hint
from src.graph.checkpointer import MongoDBSaver # Pour inspecter les checkpoints (dans une cellule ultérieure)

# Configurer le logging
LOG_LEVEL_NOTEBOOK = "INFO" # Mettre à DEBUG pour voir TOUS les logs du workflow
setup_logging(level=LOG_LEVEL_NOTEBOOK) 
logger = logging.getLogger("nb_06_e2e_test")

# --- MODIFIÉ : Vérification des prérequis pour LLMs et Embeddings utilisés par le workflow ---
logger.info(f"--- Configuration Active pour le Test de Bout en Bout ---")
# Pour les LLMs génératifs utilisés par les agents du workflow
active_llm_provider = settings.DEFAULT_LLM_MODEL_PROVIDER.lower()
logger.info(f"Fournisseur LLM génératif principal : '{active_llm_provider}'")
if active_llm_provider == "openai" and not settings.OPENAI_API_KEY:
    logger.error(f"ERREUR : Le fournisseur LLM est 'openai', mais OPENAI_API_KEY n'est pas configurée.")
elif active_llm_provider == "huggingface_api" and not settings.HUGGINGFACE_API_KEY:
    logger.error(f"ERREUR : Le fournisseur LLM est 'huggingface_api', mais HUGGINGFACE_API_KEY n'est pas configurée.")
elif active_llm_provider == "ollama" and not settings.OLLAMA_BASE_URL:
    logger.error(f"ERREUR : Le fournisseur LLM est 'ollama', mais OLLAMA_BASE_URL n'est pas configurée.")

# Pour les embeddings (utilisés par RetrievalEngine via les outils)
active_embedding_provider = settings.DEFAULT_EMBEDDING_PROVIDER.lower()
logger.info(f"Fournisseur d'Embedding (pour RAG) : '{active_embedding_provider}'")
if active_embedding_provider == "openai" and not settings.OPENAI_API_KEY:
    logger.error(f"ERREUR : Le fournisseur d'embedding est 'openai', mais OPENAI_API_KEY n'est pas configurée.")
elif active_embedding_provider == "ollama":
    if not settings.OLLAMA_BASE_URL:
        logger.error(f"ERREUR : Le fournisseur d'embedding est 'ollama', mais OLLAMA_BASE_URL n'est pas configurée.")
    if not settings.OLLAMA_EMBEDDING_MODEL_NAME:
        logger.error(f"ERREUR : Le fournisseur d'embedding est 'ollama', mais OLLAMA_EMBEDDING_MODEL_NAME n'est pas configuré.")


# Pour MongoDB (checkpointer et RAG)
if not settings.MONGO_URI:
    logger.error("ERREUR : MONGO_URI non trouvé. Le checkpointer et le RetrievalEngine (RAG) échoueront.")
# --- FIN MODIFIÉ ---

# Fonction d'affichage de l'état (se concentre sur la synthèse ou l'erreur, puis état brut)
def display_final_synthesis(final_state: dict):
    print("\n--- Synthèse Finale Produite (ou Erreur) ---")
    if not final_state:
        print("Aucun état final retourné.")
        return
    
    synthesis = final_state.get("synthesis_output")
    error_msg = final_state.get("error_message")

    if synthesis:
        print(synthesis)
    elif error_msg:
        print(f"ERREUR DANS LE WORKFLOW : {error_msg}")
    else:
        print("Aucune synthèse explicite ni message d'erreur trouvé dans les champs dédiés de l'état final.")
        print("Affichage de l'état final complet pour débogage :")
        pprint.pprint(final_state) # pprint est importé au début de la cellule
    print("------------------------------------")

### 1. Définition d'une Requête Utilisateur Complexe et Multi-Facettes

Nous allons choisir une requête qui nécessite une planification, potentiellement une recherche de nouveaux documents et une analyse de plusieurs aspects avant la synthèse.

In [ ]:
# Exemple de requête complexe :
complex_query = (
    "Provide a comprehensive overview of the use of deep reinforcement learning (DRL) for "
    "autonomous drone navigation in complex, cluttered urban environments. "
    "The overview should cover: "
    "1. Key DRL algorithms employed (e.g., PPO, SAC, DDPG variations). "
    "2. Common simulation environments and sim-to-real transfer challenges specific to this domain. "
    "3. How sensor fusion (e.g., vision, LiDAR, IMU) is typically handled in DRL policies for drones. "
    "4. Explicitly search for and include findings from any ArXiv papers published in the last 12-18 months on this topic, "
    "especially those addressing safety or obstacle avoidance. "
    "5. Summarize future research directions."
)

logger.info(f"Requête complexe pour le test de bout en bout : '{complex_query}'")

### 2. Exécution du Workflow "Cognitive Swarm"

Nous lançons le workflow avec cette requête. Le checkpointer MongoDB sauvegardera les états intermédiaires.
Nous allons observer les logs (surtout si `LOG_LEVEL_NOTEBOOK` est à `DEBUG` dans la cellule de configuration ou si la fonction `run_cognitive_swarm_v2_1` a sa propre verbosité d'événements activée).

In [ ]:
e2e_thread_id = "e2e_test_thread_" + str(uuid.uuid4())

print(f"Lancement du workflow de bout en bout pour la requête avec thread_id: {e2e_thread_id}")
print("Surveillez la console pour les logs détaillés du flux d'agents et des appels d'outils...")

final_state_e2e = None
if settings.OPENAI_API_KEY and settings.MONGO_URI:
    try:
        # La fonction run_cognitive_swarm_v2_1 affiche déjà beaucoup d'informations en streaming
        final_state_e2e = asyncio.run(run_cognitive_swarm_v2_1(complex_query, thread_id=e2e_thread_id))
    except Exception as e:
        logger.error(f"Erreur lors de l'exécution de asyncio.run(run_cognitive_swarm_v2_1): {e}", exc_info=True)
        print(f"ERREUR pendant l'exécution du workflow : {e}")
else:
    print("Clés API OpenAI ou MONGO_URI manquantes. Exécution du workflow annulée.")

# Afficher la synthèse finale (ou l'erreur)
if final_state_e2e:
    display_final_synthesis(final_state_e2e)
else:
    print("L'exécution du workflow n'a pas retourné d'état final.")

### 3. Analyse Qualitative des Sorties Intermédiaires (si `final_state_e2e` est disponible)

Si l'exécution précédente a réussi et retourné `final_state_e2e`, nous pouvons examiner certains des champs clés de cet état pour comprendre le comportement du système.

In [ ]:
if final_state_e2e and not final_state_e2e.get("error_message"):
    print("\n--- Analyse des Sorties Intermédiaires Clés ---")

    # 1. Plan de Recherche
    research_plan = final_state_e2e.get("research_plan")
    if research_plan:
        print("\n### Plan de Recherche Généré par ResearchPlannerAgent ###")
        print(research_plan)
    else:
        print("\nAucun plan de recherche explicite trouvé dans l'état final.")

    # 2. Résultats de Recherche ArXiv (si stockés explicitement)
    # Note: Dans main_workflow.py, arxiv_search_results n'est pas explicitement peuplé dans GraphState.
    # Les résultats des outils sont dans la liste `messages` (sous forme de ToolMessage).
    # On peut parcourir les messages pour trouver les ToolMessage pertinents.
    print("\n### Analyse des Messages (pour les résultats d'outils) ###")
    messages = final_state_e2e.get("messages", [])
    arxiv_tool_outputs = []
    kb_tool_outputs = []

    for msg in messages:
        if msg.type.upper() == "TOOL": # C'est un ToolMessage
            # Le nom de l'outil est dans msg.name (si défini par ToolNode, sinon dans le contenu)
            # Le contenu du ToolMessage est le résultat de l'outil.
            # Pour notre arxiv_search_tool, le résultat est une liste de dictionnaires.
            # Pour knowledge_base_retrieval_tool, c'est une liste de dictionnaires (chunks).
            
            # On pourrait essayer de deviner l'outil par le contenu ou si on avait un `name` plus explicite sur ToolMessage
            # Pour l'instant, on va juste afficher les contenus des ToolMessage.
            # On pourrait améliorer cela en ayant des noms d'outils clairs sur les ToolMessages
            # ou en regardant l'AIMessage précédent qui a fait le tool_call.
            
            # Simple affichage du contenu du ToolMessage
            # On suppose que le contenu est une chaîne ou une liste/dict sérialisable en JSON
            try:
                tool_content = json.loads(msg.content) if isinstance(msg.content, str) and (msg.content.startswith('[') or msg.content.startswith('{')) else msg.content
                # Heuristique pour différencier (très basique)
                if isinstance(tool_content, list) and tool_content and "pdf_url" in tool_content[0]:
                    print(f"\n[Résultat d'un Outil ArXiv (probable)] - {len(tool_content)} items:")
                    for item in tool_content[:2]: # Afficher les 2 premiers
                        print(f"  - Titre: {item.get('title', 'N/A')[:70]}..., PDF: {item.get('pdf_url')}")
                    arxiv_tool_outputs.append(tool_content)
                elif isinstance(tool_content, list) and tool_content and "text_chunk" in tool_content[0]:
                    print(f"\n[Résultat d'un Outil KB Retrieval (probable)] - {len(tool_content)} chunks:")
                    for item in tool_content[:1]: # Afficher le premier
                        print(f"  - Chunk ID: {item.get('chunk_id', 'N/A')}, Score: {item.get('retrieval_score', 'N/A'):.4f}")
                        print(f"    Texte: {item.get('text_chunk', '')[:150]}...")
                    kb_tool_outputs.append(tool_content)
                # else:
                #     print(f"\n[Résultat d'Outil (type inconnu ou non structuré)]:\n{str(msg.content)[:300]}...")
            except (json.JSONDecodeError, TypeError):
                # print(f"\n[Résultat d'Outil (chaîne brute)]:\n{str(msg.content)[:300]}...")
                pass # Silencieux si ce n'est pas du JSON facilement identifiable


    # 3. Résumé de l'Analyse de Documents
    doc_analysis_summary = final_state_e2e.get("document_analysis_summary")
    if doc_analysis_summary:
        print("\n### Résumé de l'Analyse de Documents (par DocumentAnalysisAgent) ###")
        print(doc_analysis_summary)
    else:
        print("\nAucun résumé d'analyse de document explicite trouvé dans l'état final (peut être intégré dans la synthèse ou les messages).")
    
    print("\n--- Fin de l'Analyse des Sorties Intermédiaires ---")

elif final_state_e2e and final_state_e2e.get("error_message"):
    print(f"L'exécution du workflow a produit une erreur, analyse des sorties intermédiaires impossible de cette manière.")
else:
    print("État final non disponible, impossible d'analyser les sorties intermédiaires.")

### 4. Inspection des Checkpoints dans MongoDB

Si le `MongoDBSaver` est actif (ce qui est le cas par défaut dans notre `main_workflow.py`), nous pouvons interroger MongoDB pour voir les états sauvegardés pour le `thread_id` de cette exécution.

In [ ]:
async def inspect_checkpoints(thread_id: str):
    logger.info(f"\n--- Inspection des Checkpoints pour Thread ID: {thread_id} ---")
    if not settings.MONGO_URI:
        logger.error("MONGO_URI non configuré. Impossible d'inspecter les checkpoints.")
        return

    checkpointer = None # Pour le bloc finally
    try:
        # Utiliser la collection configurée dans settings pour les checkpoints LangGraph
        checkpointer = MongoDBSaver(
            collection_name=settings.LANGGRAPH_CHECKPOINTS_COLLECTION
        )
        
        logger.info(f"Récupération des checkpoints pour thread_id='{thread_id}'...")
        
        # Lister tous les checkpoints pour ce thread_id
        config_for_list = {"configurable": {"thread_id": thread_id}}
        checkpoints_history = []
        async for checkpoint_tuple in checkpointer.alist(config=config_for_list):
            checkpoints_history.append(checkpoint_tuple)
        
        if not checkpoints_history:
            print(f"Aucun checkpoint trouvé pour le thread_id: {thread_id}")
            return

        print(f"Trouvé {len(checkpoints_history)} checkpoints pour le thread_id: {thread_id} (du plus récent au plus ancien):")
        
        for i, cp_tuple in enumerate(checkpoints_history[:3]): # Afficher les 3 plus récents
            print(f"\nCheckpoint #{i+1} (ts: {cp_tuple.checkpoint['id']}):")
            print(f"  Metadata: {cp_tuple.metadata}")
            print(f"  Parent ts: {cp_tuple.parent_config['configurable'].get('thread_ts') if cp_tuple.parent_config else 'None'}")
            
            # Afficher le dernier message dans ce checkpoint pour avoir une idée de l'étape
            messages_in_checkpoint = cp_tuple.checkpoint.get("channel_values", {}).get("messages", [])
            if messages_in_checkpoint:
                last_msg_in_cp = messages_in_checkpoint[-1]
                msg_type = getattr(last_msg_in_cp, 'type', 'UNKNOWN').upper()
                msg_name = getattr(last_msg_in_cp, 'name', '')
                msg_content = str(getattr(last_msg_in_cp, 'content', ''))
                print(f"  Dernier message dans ce checkpoint: [{msg_type}{' ('+msg_name+')' if msg_name else ''}]: {msg_content[:100]}...")
            else:
                print("  Aucun message dans channel_values pour ce checkpoint.")
        
        if len(checkpoints_history) > 3:
            print(f"\n... et {len(checkpoints_history) - 3} checkpoint(s) plus ancien(s).")

        # Récupérer un checkpoint spécifique (le plus récent par exemple)
        # latest_checkpoint_config = {"configurable": {"thread_id": thread_id, "thread_ts": checkpoints_history[0].checkpoint['id']}}
        # specific_checkpoint_tuple = await checkpointer.aget_tuple(latest_checkpoint_config)
        # if specific_checkpoint_tuple:
        #     print("\nDétail du checkpoint le plus récent:")
        #     pprint.pprint(specific_checkpoint_tuple.checkpoint)


    except Exception as e:
        logger.error(f"Erreur lors de l'inspection des checkpoints: {e}", exc_info=True)
        print(f"Erreur lors de l'inspection des checkpoints: {e}")
    finally:
        if checkpointer and hasattr(checkpointer, 'aclose'): # MongoDBSaver a une méthode aclose
             await checkpointer.aclose()


if final_state_e2e: # Seulement si l'exécution précédente a défini un thread_id
    asyncio.run(inspect_checkpoints(e2e_thread_id))
else:
    logger.warning("e2e_thread_id non défini ou exécution précédente échouée. Test d'inspection des checkpoints sauté.")

### 5. Discussion et Analyse Qualitative de la Synthèse Finale

Revenons à la synthèse finale produite à l'étape 2 (stockée dans `final_state_e2e['synthesis_output']`).
* La synthèse répond-elle de manière complète et précise à la requête complexe initiale ?
* Les différents aspects de la requête (algorithmes DRL, environnements de simulation, défis sim-to-real, fusion de capteurs, résultats récents d'ArXiv, directions futures) sont-ils couverts ?
* L'information est-elle bien structurée et cohérente ?
* Y a-t-il des signes d'hallucination ou des informations manquantes cruciales (en supposant que le corpus contient les informations nécessaires) ?

Cette analyse qualitative est subjective mais essentielle pour comprendre les forces et faiblesses actuelles du système. Elle peut guider les améliorations des prompts des agents, de la logique de routage, ou des stratégies RAG.

## Conclusion de ce Test de Bout en Bout

Ce notebook a permis d'exécuter le "Cognitive Swarm" sur une requête complexe, d'examiner certaines sorties intermédiaires et la synthèse finale, et de voir comment les checkpoints sont gérés.

Ce type de test approfondi est utile pour :
- Identifier les goulots d'étranglement ou les points faibles dans le flux des agents.
- Évaluer qualitativement la performance globale.
- Déboguer des comportements inattendus.
- Générer des exemples concrets pour l'évaluation quantitative (par exemple, des paires `(requête, contexte, synthèse)` pour `SynthesisEvaluator`).